In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Tue Aug 24 15:14:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    39W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation, Input, InputLayer, Flatten, Conv2D, AveragePooling2D, MaxPool2D, GlobalAveragePooling2D, SeparableConv2D, Conv1D, Conv3D, GlobalAveragePooling3D, AveragePooling3D, DepthwiseConv2D, Conv2DTranspose, Reshape
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.activations import relu
from tensorflow.keras.regularizers import L2
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow_addons.layers import StochasticDepth, GroupNormalization
from tensorflow_addons.optimizers import AdamW, RectifiedAdam

from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score

# import runai.ga.keras

In [ ]:
tf.__version__

'2.6.0'

In [ ]:
featuress = np.load('/content/drive/MyDrive/Flood/features.npy').astype(np.float32)
labelss = np.load('/content/drive/MyDrive/Flood/labels.npy').astype(np.float32)

In [ ]:
features = featuress
labels = labelss.reshape(labelss.shape[0], 512*512)

In [ ]:
IMAGE_SIZE = features.shape[1:]

In [ ]:
class SqueezeAndExcite(tf.keras.layers.Layer):
    def __init__(self, n):
        super().__init__()
        self.n = n

    def build(self, input_shape):

        self.channels = input_shape[-1]

        self.pooling = GlobalAveragePooling2D()
        self.dense_1 = Dense(self.n, activation='relu')
        self.dense_2 =  Dense(self.channels, activation='sigmoid')

    def call(self, x, training=False):

        r = self.pooling(x)
        r = self.dense_1(r, training=training)
        r = self.dense_2(r, training=training)

        r = tf.reshape(r, (-1, 1, 1, self.channels))

        return x * r
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "n": self.n,
            "channels": self.channels,
            "pooling": self.pooling,
            "dense_1": self.dense_1,
            "dense_2": self.dense_2
        })
        return config

class ResidualSEBlock(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='swish', survival_prob=1, kernel_regularizer=None,  shortcut='identity', reduction=4):
        super().__init__()
        
        self.filters = filters
        self.strides = np.array([strides, strides]) if type(strides) == int else np.array(strides)
        self.activation = activation
        self.survival_prob = survival_prob
        self.kernel_regularizer = kernel_regularizer
        self.shortcut = shortcut
        self.shortcut_mapping = None
        self.reduction = reduction

    def build(self, input_shape):

        spatial_dim = np.array([input_shape[1], input_shape[2]])
        channels = input_shape[-1]

        self.batch_norm_1 = BatchNormalization()
        self.activation_1 = Activation(self.activation)
        self.conv_1 = Conv2D(self.filters//4, 1, padding='same', kernel_regularizer=self.kernel_regularizer)
        
        self.batch_norm_2 = BatchNormalization()
        self.activation_2 = Activation(self.activation)
        self.conv_2 =  Conv2D(self.filters//4, 3, strides=self.strides, padding='same', kernel_regularizer=self.kernel_regularizer)

        self.batch_norm_3 = BatchNormalization()
        self.activation_3 = Activation(self.activation)
        self.conv_3 =  Conv2D(self.filters, 1, padding='same', kernel_regularizer=self.kernel_regularizer)

        self.squeeze_and_excite = SqueezeAndExcite(self.filters//self.reduction)
        
        if channels != self.filters or self.strides.prod() != 1: 
          if self.shortcut == 'identity':
            self.projection = AveragePooling2D(pool_size=self.strides, strides=self.strides, padding='same')
            self.shortcut_mapping = lambda x : tf.pad(self.projection(x), [[0, 0], [0, 0], [0, 0], [0, self.filters - channels]])
          if self.shortcut == 'projection':
            self.projection = Conv2D(self.filters, 1, strides=self.strides, padding='same')
            self.shortcut_mapping = lambda x : self.projection(x)
        else:
          self.projection = None
          self.shortcut_mapping = lambda x : x
        
        if self.survival_prob != 1:
          self.stochastic_depth = StochasticDepth(self.survival_prob)
        else:
          self.stochastic_depth = None

    def call(self, x, training=False):
        
        r = self.batch_norm_1(x, training=training)
        r = self.activation_1(r)
        r = self.conv_1(r, training=training)
        
        r = self.batch_norm_2(r, training=training)
        r = self.activation_2(r)
        r = self.conv_2(r, training=training)

        r = self.batch_norm_3(r, training=training)
        r = self.activation_3(r)
        r = self.conv_3(r, training=training)

        r = self.squeeze_and_excite(r)

        x = self.shortcut_mapping(x)

        if self.survival_prob != 1:
          return self.stochastic_depth([x, r], training=training)

        return x + r

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "strides": self.strides,
            "activation": self.activation,
            "survival_prob": self.survival_prob,
            "kernel_regularizer": self.kernel_regularizer,
            "shortcut": self.shortcut,
            "projection": self.projection,
            "shortcut_mapping": self.shortcut_mapping,
            "reduction": self.reduction,
            "batch_norm_1": self.batch_norm_1,
            "activation_1": self.activation_1,
            "conv_1": self.conv_1,
            "batch_norm_2": self.batch_norm_2,
            "activation_2": self.activation_2,
            "conv_2": self.conv_2,
            "batch_norm_3": self.batch_norm_3,
            "activation_3": self.activation_3,
            "conv_3": self.conv_3,
            "squeeze_and_excite": self.squeeze_and_excite,
            "stochastic_depth": self.stochastic_depth,
        })
        return config
    
class ResidualSEBlockT(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='swish', survival_prob=1, kernel_regularizer=None,  shortcut='identity', reduction=4):
        super().__init__()
        
        self.filters = filters
        self.strides = np.array([strides, strides]) if type(strides) == int else np.array(strides)
        self.activation = activation
        self.survival_prob = survival_prob
        self.kernel_regularizer = kernel_regularizer
        self.shortcut = shortcut
        self.shortcut_mapping = None
        self.reduction = reduction

    def build(self, input_shape):

        spatial_dim = np.array([input_shape[1], input_shape[2]])
        channels = input_shape[-1]

        self.batch_norm_1 = BatchNormalization()
        self.activation_1 = Activation(self.activation)
        self.conv_1 = Conv2D(self.filters//4, 1, padding='same', kernel_regularizer=self.kernel_regularizer)
        
        self.batch_norm_2 = BatchNormalization()
        self.activation_2 = Activation(self.activation)
        self.conv_2 =  Conv2DTranspose(self.filters//4, 3, strides=self.strides, padding='same', kernel_regularizer=self.kernel_regularizer)

        self.batch_norm_3 = BatchNormalization()
        self.activation_3 = Activation(self.activation)
        self.conv_3 =  Conv2D(self.filters, 1, padding='same', kernel_regularizer=self.kernel_regularizer)

        self.squeeze_and_excite = SqueezeAndExcite(self.filters//self.reduction)
        
        if channels != self.filters or self.strides.prod() != 1: 
          if self.shortcut == 'identity':
            self.projection = AveragePooling2D(pool_size=self.strides, strides=self.strides, padding='same')
            self.shortcut_mapping = lambda x : tf.pad(self.projection(x), [[0, 0], [0, 0], [0, 0], [0, self.filters - channels]])
          if self.shortcut == 'projection':
            self.projection = Conv2DTranspose(self.filters, 1, strides=self.strides, padding='same')
            self.shortcut_mapping = lambda x : self.projection(x)
        else:
          self.shortcut_mapping = lambda x : x
        
        if self.survival_prob != 1:
          self.stochastic_depth = StochasticDepth(self.survival_prob) 

    def call(self, x, training=False):
        
        r = self.batch_norm_1(x, training=training)
        r = self.activation_1(r)
        r = self.conv_1(r, training=training)
        
        r = self.batch_norm_2(r, training=training)
        r = self.activation_2(r)
        r = self.conv_2(r, training=training)

        r = self.batch_norm_3(r, training=training)
        r = self.activation_3(r)
        r = self.conv_3(r, training=training)

        r = self.squeeze_and_excite(r)

        x = self.shortcut_mapping(x)

        if self.survival_prob != 1:
          return self.stochastic_depth([x, r], training=training)

        return x + r

class ResidualSEBlock2(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='swish', survival_prob=1, kernel_regularizer=None,  shortcut='identity', reduction=4):
        super().__init__()
        
        self.filters = filters
        self.strides = np.array([strides, strides]) if type(strides) == int else np.array(strides)
        self.activation = activation
        self.survival_prob = survival_prob
        self.kernel_regularizer = kernel_regularizer
        self.shortcut = shortcut
        self.shortcut_mapping = None
        self.reduction = reduction

    def build(self, input_shape):

        spatial_dim = np.array([input_shape[1], input_shape[2]])
        channels = input_shape[-1]

        self.batch_norm_1 = BatchNormalization()
        self.activation_1 = Activation(self.activation)
        self.conv_1 = Conv2D(self.filters, 3, strides=self.strides, padding='same', kernel_regularizer=self.kernel_regularizer)

        self.batch_norm_3 = BatchNormalization()
        self.activation_3 = Activation(self.activation)
        self.conv_3 =  Conv2D(self.filters, 3, padding='same', kernel_regularizer=self.kernel_regularizer)

        self.squeeze_and_excite = SqueezeAndExcite(self.filters//self.reduction)
        
        if channels != self.filters or self.strides.prod() != 1: 
          if self.shortcut == 'identity':
            self.projection = AveragePooling2D(pool_size=self.strides, strides=self.strides, padding='same')
            self.shortcut_mapping = lambda x : tf.pad(self.projection(x), [[0, 0], [0, 0], [0, 0], [0, self.filters - channels]])
          if self.shortcut == 'projection':
            self.projection = Conv2D(self.filters, 1, strides=self.strides, padding='same')
            self.shortcut_mapping = lambda x : self.projection(x)
        else:
          self.projection = None
          self.shortcut_mapping = lambda x : x
        
        if self.survival_prob != 1:
          self.stochastic_depth = StochasticDepth(self.survival_prob)
        else:
          self.stochastic_depth = None

    def call(self, x, training=False):
        
        r = self.batch_norm_1(x, training=training)
        r = self.activation_1(r)
        r = self.conv_1(r, training=training)

        r = self.batch_norm_3(r, training=training)
        r = self.activation_3(r)
        r = self.conv_3(r, training=training)

        r = self.squeeze_and_excite(r)

        x = self.shortcut_mapping(x)

        if self.survival_prob != 1:
          return self.stochastic_depth([x, r], training=training)

        return x + r

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "strides": self.strides,
            "activation": self.activation,
            "survival_prob": self.survival_prob,
            "kernel_regularizer": self.kernel_regularizer,
            "shortcut": self.shortcut,
            "projection": self.projection,
            "shortcut_mapping": self.shortcut_mapping,
            "reduction": self.reduction,
            "batch_norm_1": self.batch_norm_1,
            "activation_1": self.activation_1,
            "conv_1": self.conv_1,
            "batch_norm_2": self.batch_norm_2,
            "activation_2": self.activation_2,
            "conv_2": self.conv_2,
            "batch_norm_3": self.batch_norm_3,
            "activation_3": self.activation_3,
            "conv_3": self.conv_3,
            "squeeze_and_excite": self.squeeze_and_excite,
            "stochastic_depth": self.stochastic_depth,
        })
        return config

class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='swish', survival_prob=1, kernel_regularizer=None,  shortcut='identity', first=False):
        super().__init__()
        
        self.filters = filters
        self.strides = np.array([strides, strides]) if type(strides) == int else np.array(strides)
        self.activation = activation
        self.survival_prob = survival_prob
        self.kernel_regularizer = kernel_regularizer
        self.shortcut = shortcut
        self.shortcut_mapping = None
        self.first = first

    def build(self, input_shape):

        spatial_dim = np.array([input_shape[1], input_shape[2]])
        channels = input_shape[-1]

        if not self.first:
          self.batch_norm_1 = BatchNormalization()
          self.activation_1 = Activation(self.activation)
        self.conv_1 = Conv2D(self.filters, 3, strides=self.strides, padding='same', kernel_regularizer=self.kernel_regularizer)
        
        self.batch_norm_2 = BatchNormalization()
        self.activation_2 = Activation(self.activation)
        self.conv_2 =  Conv2D(self.filters, 3, padding='same', kernel_regularizer=self.kernel_regularizer)
        
        if channels != self.filters or self.strides.prod() != 1: 
          if self.shortcut == 'identity':
            self.projection = AveragePooling2D(pool_size=self.strides, strides=self.strides, padding='same')
            self.shortcut_mapping = lambda x : tf.pad(self.projection(x), [[0, 0], [0, 0], [0, 0], [0, self.filters - channels]])
          if self.shortcut == 'projection':
            self.projection = Conv2D(self.filters, 1, strides=self.strides, padding='same')
            self.shortcut_mapping = lambda x : self.projection(x)
        else:
          self.shortcut_mapping = lambda x : x
        
        if self.survival_prob != 1:
          self.stochastic_depth = StochasticDepth(self.survival_prob) 

    def call(self, x, training=False):
        
        r = x

        if not self.first:
          r = self.batch_norm_1(r, training=training)
          r = self.activation_1(r)
        r = self.conv_1(r, training=training)
        
        r = self.batch_norm_2(r, training=training)
        r = self.activation_2(r)
        r = self.conv_2(r, training=training)

        x = self.shortcut_mapping(x)

        if self.survival_prob != 1:
          return self.stochastic_depth([x, r], training=training)

        return x + r

In [ ]:
num_models = 6

In [ ]:
class Augmentation(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()
    self.flip = tf.keras.layers.RandomFlip()
  
  def call(self, data, training=False):
    if not training:
      return data
    
    x, y = data
    rotation = tf.cast(tf.random.uniform((1,))[0] * 4, tf.int32)
    
    x = tf.image.rot90(x, rotation)
    y = tf.image.rot90(y, rotation)

    x = self.flip(x)
    y = self.flip(y)

    return x, y

In [ ]:
def apply_transform(x, rotation, h_flip, v_flip):
  x = tf.image.rot90(x, rotation)
  if h_flip:
    x = tf.image.flip_left_right(x)
  if v_flip:
    x = tf.image.flip_up_down(x)
  return x

class Model2(Model):

    @tf.function
    def train_step(self, data):
        
        x, y = data

        y = tf.expand_dims(y, axis=-1)

        X = [x]
        Y = [y]
        for rotation in range(1, 4):
          for h_flip in range(2):
            for v_flip in range(2):
              X.append(apply_transform(x_train, rotation, h_flip, v_flip))
              Y.append(apply_transform(y_train, rotation, h_flip, v_flip))
        x = tf.concatenate(X, axis=0)
        y = tf.concatenate(Y, axis=0)

        y = tf.reshape(y, [-1, 262144])

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

In [ ]:
def unet_encoder(n, filters, x, pooling=True, reduce_dim=False):
    if reduce_dim:
      x = Conv2D(reduce_dim, 3, padding='same')(x)
    for i in range(n):
        x = ResidualSEBlock(filters)(x)
    if pooling:
      p = MaxPool2D(padding='same')(x)
      return x, p
    else:
      return x

def unet_decoder(n, filters, x, r):
    x = Conv2DTranspose(filters, 3, strides=2, padding='same')(x)
    x = x + r
    for i in range(n):
      x = ResidualSEBlock(filters)(x)
    return x

In [ ]:
factor = 4

x = inp_x = Input(IMAGE_SIZE)

# y = inp_y = Input((512, 512))
# y = tf.expand_dims(y, axis=-1)

# x, y = RandomAugmentation()([x, y])

# y = tf.reshape(y, [-1, 262144])

x = Conv2D(2, 3, strides=2, padding='same')(x)
x = Conv2D(2, 3, strides=2, padding='same')(x)

x00, p00 = unet_encoder(3, int(64/factor), x)
x10, p10 = unet_encoder(4, int(128/factor), p00)
x20, p20 = unet_encoder(5, int(256/factor), p10)
x30, p30 = unet_encoder(6, int(512/factor), p20)

x40 = unet_encoder(7, int(1024/factor), p30, pooling=False)

x01 = unet_decoder(3, int(64/factor), x10, x00)

x11 = unet_decoder(4, int(128/factor), x20, x10)

x02 = unet_decoder(3, int(64/factor), x11, x00 + x01)

x21 = unet_decoder(5, int(256/factor), x30, x20)

x12 = unet_decoder(4, int(128/factor), x21, x10 + x11)

x03 = unet_decoder(3, int(64/factor), x12, x00 + x01 + x02)

x31 = unet_decoder(6, int(512/factor), x40, x30)

x22 = unet_decoder(5, int(256/factor), x31, x20 + x21)

x13 = unet_decoder(4, int(128/factor), x22, x10 + x11 + x12)

x04 = unet_decoder(3, int(64/factor), x13, x00 + x01 + x02 + x03)


x = x04

x = BatchNormalization()(x)
x = Activation('swish')(x)
x = Conv2DTranspose(1, 3, strides=2, padding='same')(x)
x = Conv2DTranspose(1, 3, strides=2, padding='same')(x)
x = Conv2D(1, 3, activation='sigmoid', padding='same')(x)
x = Flatten()(x)

models = [Model2(inputs=inp_x, outputs=x)]
models[0].summary()

Model: "model2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 2) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 2)  38          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 2)  38          conv2d[0][0]                     
__________________________________________________________________________________________________
residual_se_block (ResidualSEBl (None, 128, 128, 16) 428         conv2d_1[0][0]                   
_____________________________________________________________________________________________

In [ ]:
class AdjustedBinaryCrossentropy(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    self.loss = tf.keras.losses.BinaryCrossentropy()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    return self.loss(y_true, y_pred, sample_weight=sample_weight)

class LogCoshDice(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    # self.loss = tf.keras.losses.LogCosh()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    dice = 1 - tf.reduce_mean((2 * tf.reduce_sum(y_true * y_pred, axis=-1) + 1)/ (tf.reduce_sum(y_true, axis=-1) + tf.reduce_sum(y_pred, axis=-1) + 1))
    return tf.math.log(tf.cosh(dice) + 1e-7) 

class SoftDice(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    # self.loss = tf.keras.losses.LogCosh()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    return tf.reduce_mean(1 - (2 * tf.reduce_sum(y_true * y_pred, axis=-1) + 1) / (tf.reduce_sum(y_true**2, axis=-1) + tf.reduce_sum(y_pred**2, axis=-1) + 1))

class PowerJaccard(tf.keras.losses.Loss):
  def __init__(self, p):
    super().__init__()
    self.p = p

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    inter = tf.reduce_sum(y_true * y_pred, axis=-1)
    return tf.reduce_mean(1 - (inter + 1e-7)/ (tf.reduce_sum(y_true**self.p, axis=-1) + tf.reduce_sum(y_pred**self.p, axis=-1) - inter + 1e-7))

class AdjustedBinaryCrossentropyAndDiceCoeff(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    self.loss1 = tf.keras.losses.BinaryCrossentropy()
    self.loss2 = LogCoshDice()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    return (self.loss1(y_true, y_pred) + self.loss2(y_true, y_pred)) / 2

class AdjustedBinaryCrossentropyAndDiceCoeff2(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    self.loss1 = tf.keras.losses.BinaryCrossentropy()
    self.loss2 = SoftDice()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    return (self.loss1(y_true, y_pred) + self.loss2(y_true, y_pred)) / 2

class AdjustedBinaryCrossentropyAndDiceCoeff3(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    self.loss1 = tf.keras.losses.BinaryCrossentropy()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    inter = tf.reduce_sum(y_true * y_pred)
    loss2 = inter / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + inter + 1e-7) / (16 * 512 * 512)
    return (self.loss1(y_true, y_pred) + loss2) / 2

def jaccard(y_true, y_pred):
  mask = tf.cast(y_true != 255, tf.float32)
  y_true = tf.cast((y_true * mask) > 0.5, tf.float32)
  y_pred =  tf.cast((y_pred * mask) > 0.5, tf.float32)
  
  n_null = tf.reduce_sum(tf.cast(y_true == 255, tf.float32), axis=1)

  inter = tf.reduce_sum(tf.cast(y_true == y_pred, tf.float32), axis=1)
  return (inter - n_null) / (2*512*512 - inter - n_null)

In [ ]:
x_train, y_train = features, labelss
y_train2 = y_train.reshape(x_train.shape[0], 512*512)

In [ ]:
batch_size = 2
decay_epochs = 30
epochs = 80

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train2)).batch(batch_size)

In [ ]:
for i, model in enumerate(models):

  opt = Adam(tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=(x_train.shape[0]//batch_size)*decay_epochs, decay_rate=0.1, staircase=True))
  loss = AdjustedBinaryCrossentropy()
  # loss = LogCoshDice()
  # loss = AdjustedBinaryCrossentropyAndDiceCoeff()

  model.compile(optimizer=opt, loss=loss, metrics=[jaccard])
  model.fit(train_dataset, batch_size=batch_size, epochs=epochs , validation_data=test_dataset)

  # model.save(f'/content/drive/MyDrive/Flood/model_{i+1+num_saved}')

Epoch 1/80


In [ ]:
model.save_weights('/content/drive/MyDrive/Flood/model_full_rotation_weights')